# 1.1 Is Unique

Implement an algorihtm to determine if a string has all unique characters. What if we cannot use additional data structures?

## Answer - Hash Table / Dictionary

* uniqueness checking = use a hash table
* hash a character. if it's already in the hash table then we've seen it before and return false. otherwise return true
* runtime:
  * hash a character: O(1)
  * n characters -> O(n)
  * do this once on a string

In [1]:
from collections import defaultdict

def is_unique(string):
    d = defaultdict(int)
    for character in string:
        index = hash(character)
        if d[index]:
            return False
        
        d[index] = 1
    return True

# tests
assert(is_unique('a'))
assert(is_unique(''))
assert(is_unique('abc'))
assert(not is_unique('aa'))
assert(not is_unique('hotelmotelholidayinn'))

## Can we do better?

*Time*: No. At the very least each character needs to be visited.

*Memory*: Maybe.

## Book Answer

* Uses a fixed length array O(n)
* Also uses a bit vector of length 
* No additional data strctures:
  * O(n^2) --- compare everything with everything after it
  * O(n log(n)) + O(n) --- sort and then compare neighbors
  
### Side Note: Python's Sorting Algorithm

Python uses [timsort](https://en.wikipedia.org/wiki/Timsort)
* Time:
  * $O(n)$ best
  * $O(n \log(n))$ average
  * $O(n \log(n))$ worst
* Memory: $O(n)$

## 1.2 Check Permutation

Given two strings, write a method to decide if one is a permutation of the other.

## Solution

Can also be solved by a hash table: keys = h(characters), values = #occurrences in `s1`. After processing `s1` you next process the characters `c` in `s2`.
* if `c` is not in the hash table: return False
* if `c` is in the hash table:
    if d[`c`] == 0: return false
    else: d[`c`] -= 1

**Complexity**
* Time: $2n = O(n)$
  * n hash stores ($O(1)$ each) in `s1`, n hashes in `s2`
* Memory: $n = O(n)$
  * best case: $1 = O(1)$ (both strings contain only one letter)
  * worst case: $n = O(n)$. ($n$ different letters)

In [2]:
from collections import defaultdict

def is_permutation(s1, s2):
    # first check if s1 and s2 are same length
    if len(s1) != len(s2):
        return False
    
    d = defaultdict(int)
    
    # store s1
    for c in s1:
        d[c] += 1
        
    # compare with s2
    for c in s2:
        if not d[c]:
            return False
        else:
            d[c] -= 1
    return True

# tests
assert(is_permutation('', ''))
assert(is_permutation('a', 'a'))
assert(is_permutation('ab', 'ba'))
assert(is_permutation('abc', 'cab'))
assert(not is_permutation('abc', 'caa'))
assert(not is_permutation('abc', ''))
assert(not is_permutation('', 'abc'))


### Can we reduce constant factor from subtraction?

We can use a null stack and pop.

* Memory: $O(n)$
  * worst case: $n = O(n)$ (even if there is only one letter you need $n$ list elements to track the occurrences
  * best case: $n$
  
However, the memory operations to create a stack node may take longer than integer arithmetic. (1 cycle for addition)

In [3]:
from collections import defaultdict

def is_permutation(s1, s2):
    # first check if s1 and s2 are same length
    if len(s1) != len(s2):
        return False
    
    d = defaultdict(list)
    
    # store s1
    for c in s1:
        d[c].append(None)
        
    # compare with s2
    for c in s2:
        if not d[c]:
            return False
        else:
            d[c].pop()
    return True

# tests
assert(is_permutation('', ''))
assert(is_permutation('a', 'a'))
assert(is_permutation('ab', 'ba'))
assert(is_permutation('abc', 'cab'))
assert(not is_permutation('abc', 'caa'))
assert(not is_permutation('abc', ''))
assert(not is_permutation('', 'abc'))



# 1.9 String Rotation

Assume you have a method `isSubstring` which checks is one word is a substring of antoher. Given two strings, `s1`, and `s2`, write code to check if `s2` is a rotation of `s1` using only one call to `isSubstring` (e.g. `waterbottle` is a rotation of `erbottlewat`.)

## Attempt

We might first assume that we can take, say, the first four letters of `waterbottle` and check if it's a substring of `s2`. However, this wouldn't work since `wate` is not a substring of `s2`.

Consider the first character of `s1` and find all the positions in `s2` where the character occurs. Store indices in queue.
* $O(n)$ operation

Now take the second character of `s1`. dequeue an index `i` and check if `i+1 % n` of `s2` is equal to the character. If so, enqueue `i+1 % n`. Otherwise, discard.
* $O(n)$ operation

Problem: we need to keep track of old and new parts. Use two queues?

Repeat: need to do this $n$ times so this is an $O(n^2)$ solution.

In [4]:
def is_rotation(s1, s2):
    n = len(s1)
    if n != len(s2):
        return False
    
    # get the indices in s2 matching s1[0]
    current = [i for i in range(n) if s1[0] == s2[i]]
    for k in range(1,n):
        # get the indices in s2 matching s1[i]
        next = [(i+1) % n for i in current if s1[k] == s2[(i+1) % n]]
        current = next
        
        if current == []:
            return False
        
    return True
    
    
# tests
assert(is_rotation('abc', 'abc'))
assert(is_rotation('abc', 'cab'))
assert(is_rotation('waterbottle', 'erbottlewat'))
assert(is_rotation('aaabbbccc', 'aabbbccca'))

assert(not is_rotation('abc', 'acb'))

def shift_check(s1):
    n = len(s1)
    for k in range(n):
        s2 = s1[:n] + s1[n:]
        assert(is_rotation(s1,s2))
        
shift_check('banana')
shift_check('abracadabra')
shift_check('puppymonkeybaby')
shift_check('aaaaaaaaaaaaah')

The worst case is when the string consists of `n` equal characters.

## Anything Faster?

Hint: try concatenating `s2` with itself.

## Solution

if `s1` splits into two parts `x` and `y` such that `xy = s1` and `yx = s2` then we're done. However, `yx` is always a substring of `xyxy = s1s1`. So the solution is to do

`is_substring(s1s1, s2)`

# Random Problem - p. 67

Given an array of distinct integer values, count the number of pairs of integers that have difference `k`. For example, given `{1, 7, 5, 9, 2, 12, 3}` and `k = 2` there are four pairs with difference 2: `(1,3), (3,5), (5,7), (7,9)`.

# My Solution

Easier to determine differences after sorting: check current and next. 
* sorting: $O(n \log n)$
* checking: single forward scan $O(n)$
* ==> $O(n \log n)$

Is there an approach faster than $O(n \log n)$?

`a - b = k iff h(a-b) = h(k) iff h(a) == h(k+b)`

One approach is to take the array, `arr`, and the shfited array `arr + k`. hash the elements of `arr` and the hash the elements of `arr + k`. If there is collision then we found a pair.

Does it matter that `a > b`

`a - b < 0 ==> b - a > 0 ==> b - a == k iff b == k + a iff h(b) = h(k+a)`

but we end up computing h(b) anyway from `arr` and `h(k+a)` from `arr + k`. So we shouldn't have to check for which is greater or not.

**Complexity**

* Total 2n hashes computed: Time = $O(n)$
* Only n hashes stored: Mem = $O(n)$

In [5]:
from collections import defaultdict

def f(arr, k):
    d = defaultdict(int)
    num_pairs = 0
    
    for a in arr:
        d[hash(a)] = 1
        
    for b in arr:
        if d[hash(b+k)]:
            num_pairs += 1
            
    return num_pairs

arr = [1, 7, 5, 9, 2, 12, 3]
print f(arr, 2)

4


# Random Problem - p. 70

Given a smaller string `s` and a bigger string `b` design an algorithm to find all permutations of the shorter string within the longer one. Print the location of each permutation.

## My Solution

The wrong way: create a list of all permutations of `s` and perform a linear search on `b` for each one. $O(|b|\;|s|!)$

Create a hash table with `character of s` |-> `number of occurrences`. Then, starting at the first character of `b` scan up to `|s|` spaces and decrement the corresponding number of occurences:
* if there is a key error, break and move to one past the key error (any subsequent words won't work until afterwards)
* if by the time we reach |s| increment by one and repeat
* otherwise, if we reach |s| and the number of occurrences is now zero we add the index to the list

example

`s = cat`
`b = tacocat`

output

`0,4`

**Analysis**

* Creation of `s` hash table: $O(s)$
* Scanning each `|s|` block of text in `b`: $O(s)$
* Number of substrings we need to check: $O(b-s)$
* Total: $O(s(b-s)) = O(sb)$
* Skipping trick (constant factor)

In [6]:
from collections import defaultdict

def all_permutations(s, b):
    d = defaultdict(int)
    
    # build the s-character dictionary
    for char in s:
        d[char] += 1
        
    # start scanning
    indices = []
    n = len(b) - len(s) + 1
    s = len(s)
    for i in range(n):
        # we want a copy since Python assigns dictionaries by reference
        d_pass = d.copy()
        
        is_substring = True
        for j in range(s):
            char = b[i+j]
            if d_pass[char] > 0:
                d_pass[char] -= 1
            else:
                is_substring = False
                break
                
        if is_substring:
            indices.append(i)
            
    return indices
            
# tests

assert(all_permutations('a', 'a') == [0])
assert(all_permutations('a', 'aa') == [0,1])
assert(all_permutations('a', 'aaa') == [0,1,2])
assert(all_permutations('a', 'aba') == [0,2])
assert(all_permutations('ab', 'abcba') == [0,3])
assert(all_permutations('bc', 'abcba') == [1,2])

assert(all_permutations('cat', 'tacocat') == [0,4])



## Alternate Approach

I'm thinking we might be able to get this down to $O(b-s) + O(s)$ by using a "scanner approach".

`s = bat`

`b = otbafabstab`

`otbafabstab` should output `[1, 8]`

* intialize on first s: count the number of matching letters

no. this will still be O(s(b-s)) time. 